# The code below loads the new VERA model for the emotion extraction

In [30]:
%load_ext autoreload
%autoreload 2

import Structures
import EmotionExtractor_tf

#This one works with theano or tensorflow backend... see which one you prefer, I suggest TF.
em = EmotionExtractor_tf.EmotionExtractor('baseline_context5_conv_simple2.weights', 'mean_std.csv')





The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Structures.py:148: UserWarning: Update your `Model` call to the Keras 2 API: `Model(outputs=Softmax.0, inputs=[/input_11...)`
  model = Model(input=[encoding_input], output=output)


# The code below is for the case in which you are analysing Techcrunch Pitches

In [3]:
import os                                                                                                             
from pydub import AudioSegment
import json
import pandas as pd
import pydub.exceptions
from natsort import natsorted, ns
#mykeys= dict_files.keys()


#This is for Techcrunch.
def list_files_dir(dir):                                                                                                  
    r = {}                                                                                                            
    subdirs = [x[0] for x in os.walk(dir)]                                                                            
    for subdir in subdirs:
        
        files = os.walk(subdir).next()[2]                                                                             
        if (len(files) > 0):
            r[subdir]= []
            for file in files: 
                    r.get(subdir).append(subdir + "\\" + file)
                    #print subdir + "\\" + file
            r[subdir] = natsorted(r.get(subdir), key=lambda y: y.lower())

    return r

#Techcrunch.
dict_files = list_files_dir('D://DataElisa/PitchesAudio2/audio_final/')

#Earning Calls



#print(dict_files)





In [21]:
dirnames1= dict_files.keys()


for i in dirnames1:
    if i.find("3.0secSplit")==-1: 
        print(i)
        dict_files.pop(i,None)
#cleanup

dirnames2 = dict_files.keys()


dirnames=dirnames2

# Techcrunch Pitches Analysis

In [12]:
#getting them 3 by 3

import traceback
import pickle

len_sequence = 5
byte_depth =16

dirnames= dict_files.keys()
dictionary_for_jason = {}

#dictionary_for_jason = pickle.load( open( "techcrunch3.pkl", "rb" ))

iterator_idx = 0

for index_name in range(0, len(dirnames)):
    print index_name
    name = dirnames[index_name]
    dictionary_for_jason[name] = []
    filenames=dict_files.get(name)
    
    for j in range(len_sequence,len(filenames)+len_sequence,len_sequence):
        #try:
        totfilenames =filenames[j-len_sequence:j]
        song = None
        for segment in totfilenames:
            if song==None:
                song= AudioSegment.from_mp3(segment)
            else:
                segement_add= AudioSegment.from_mp3(segment)
                song=song.append(segement_add)
        result =  em.split_song_get_emotion(song,len_sequence,16)
        #print result
        dictionary_for_jason[name].append(result)
    
    if iterator_idx % 10 ==0 or index_name==len(dirnames)-1:    
        pickle.dump( dictionary_for_jason, open( "techcrunch_" + str(iterator_idx) +".pkl", "wb" ) )   
        dictionary_for_jason = {}
    
    iterator_idx += 1
    
    
        #except:
        #    traceback.print_exc()
        #    print("Not Decodable")
        #    return


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60


In [9]:
#Picking Up Earning Calls splits



# Earning Calls Emotion Analysis.

In [4]:
import os                                                                                                             
from pydub import AudioSegment
import json
import pandas as pd
import pydub.exceptions
from natsort import natsorted, ns

# This is for Earning Calls.
def list_files(dir):                                                                                                  
    r = []                                                                                                            
    subdirs = [x[0] for x in os.walk(dir)]                                                                            
    for subdir in subdirs:                                                                                            
        files = os.walk(subdir).next()[2]                                                                             
        if (len(files) > 0):                                                                                          
            for file in files: 
                if("qna_f" in file):
                    r.append(subdir + "\\" + file)
                    #print subdir + "\\" + file
    return r

#list all the files of all the companies analysed, for example:
list_my_files= list_files("D:\\DataElisa\\Nokia")
print(len(list_my_files))
#Adding SES:
list_my_files.extend(list_files("D:\\DataElisa\\SES"))
print(len(list_my_files))
#Adding BT:
list_my_files.extend(list_files("D:\\DataElisa\\BT"))
print(len(list_my_files))



7160
12752
24826


# Everything is saved in the emovec.json.

In [31]:
import sys, traceback

len_sequence = 5 

#length of the sequence. In VERA we have 15 seconds length, 
#I would suggest to rely on long sentences for the emotion, short sentences are
#unreliable/noise for this model.

byte_depth = 8 #this has to deal with the precision of the recorded file.
#in this case 16 does not work.

result_all = pd.DataFrame()
to_json = []
count=0
for i in list_my_files:
    try:
        song = AudioSegment.from_mp3(i)
        result =  em.split_song_get_emotion(song,len_sequence, byte_depth)

        result["filename"] = i
    
        dict_to_append = result.to_dict('record')
        #print(dict_to_append)
    
        to_json.append(dict_to_append[0])
        
        print(count)
        if count % 100 ==0:
            with open('emovec_earning_calls.json', 'w') as outfile:
                json.dump(to_json, outfile)
                outfile.flush()
                outfile.close()
        if count== len(list_my_files)-1:
            with open('emovec_earning_calls.json', 'w') as outfile:
                json.dump(to_json, outfile)
                outfile.flush()
                outfile.close()
        count += 1
    except:
        print("The File is Not Decodable")
        traceback.print_exc(file=sys.stdout)
        #break
        
    #print result.to_dict('list')
    #print json.dumps(to_json)
    #print result_all.to_json()

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
Not Decodable
Traceback (most recent call last):
  File "<ipython-input-31-b2fc93b6cf59>", line 11, in <module>
    song = AudioSegment.from_mp3(i)
  File "D:\Python2.7\lib\site-packages\pydub\audio_segment.py", line 514, in from_mp3
    return cls.from_file(file, 'mp3')
  File "D:\Python2.7\lib\site-packages\pydub\audio_segment.py", line 501, in from_file
    raise CouldntDecodeError("Decoding failed. ffmpeg returned error code: {0}\n\nOutput from ffmpeg/avlib:\n\n{1}".format(p.returncode, p_err))
CouldntDecodeError: Decoding failed. ffmpeg returned error code: 1

Output from ffmpeg/avlib:

ffmpeg version 2.7 Copyright (c) 2000-2015 the FFmpeg developers
  built with gcc 4.9.2 (GCC)
  configuration: --enable-gpl --enable-version3 --disable-w32threads --enable-avisynth --enable-bzlib --enable-fontconfig --enable-frei0r --enable-gnutls --enabl

In [9]:
with open('jsn_nokia.txt', 'w') as outfile:
                json.dump(to_json, outfile)
                outfile.flush()
                outfile.close()

In [6]:
list_my_files

['D:\\DataElisa\\SES\\sesq113qna\\mp3\\sesq113qna_f000001.mp3',
 'D:\\DataElisa\\SES\\sesq113qna\\mp3\\sesq113qna_f000002.mp3',
 'D:\\DataElisa\\SES\\sesq113qna\\mp3\\sesq113qna_f000003.mp3',
 'D:\\DataElisa\\SES\\sesq113qna\\mp3\\sesq113qna_f000004.mp3',
 'D:\\DataElisa\\SES\\sesq113qna\\mp3\\sesq113qna_f000005.mp3',
 'D:\\DataElisa\\SES\\sesq113qna\\mp3\\sesq113qna_f000006.mp3',
 'D:\\DataElisa\\SES\\sesq113qna\\mp3\\sesq113qna_f000007.mp3',
 'D:\\DataElisa\\SES\\sesq113qna\\mp3\\sesq113qna_f000008.mp3',
 'D:\\DataElisa\\SES\\sesq113qna\\mp3\\sesq113qna_f000009.mp3',
 'D:\\DataElisa\\SES\\sesq113qna\\mp3\\sesq113qna_f000010.mp3',
 'D:\\DataElisa\\SES\\sesq113qna\\mp3\\sesq113qna_f000011.mp3',
 'D:\\DataElisa\\SES\\sesq113qna\\mp3\\sesq113qna_f000012.mp3',
 'D:\\DataElisa\\SES\\sesq113qna\\mp3\\sesq113qna_f000013.mp3',
 'D:\\DataElisa\\SES\\sesq113qna\\mp3\\sesq113qna_f000014.mp3',
 'D:\\DataElisa\\SES\\sesq113qna\\mp3\\sesq113qna_f000015.mp3',
 'D:\\DataElisa\\SES\\sesq113qna\\mp3\\s

In [20]:
def list_files2(dir):                                                                                                  
    r = []                                                                                                            
    subdirs = [x[0] for x in os.walk(dir)]                                                                            
    for subdir in subdirs:                                                                                            
        files = os.walk(subdir).next()[2]                                                                             
        if (len(files) > 0):                                                                                          
            for file in files: 
                 if(".wav" in file):
                    r.append(subdir + "\\" + file)
                    #print subdir + "\\" + file
    return r


list_my_files2= list_files2("C:\\Users\\Stefano\\workspace\\EmodashSamples\\SamplesStandard\\AudioData\\AudioData")

for i in list_my_files2:
    print i
    if "h" in i:
        song = AudioSegment.from_mp3(i)
        print em.split_song_get_emotion(song,5,8)


C:\Users\Stefano\workspace\EmodashSamples\SamplesStandard\AudioData\AudioData\DC\a01.wav
          Anger   Disgust      Fear     Happiness   Neutral       Sadness  \
0  5.339498e-09  0.802652  0.000090  5.129811e-07  0.077196  8.965202e-09   
1  1.472221e-06  0.000002  0.000002  2.468351e-09  0.009266  6.772124e-06   

   Surprise  
0  0.120061  
1  0.990722  
C:\Users\Stefano\workspace\EmodashSamples\SamplesStandard\AudioData\AudioData\DC\a02.wav
          Anger   Disgust      Fear     Happiness   Neutral       Sadness  \
0  1.026874e-11  0.863464  0.000134  5.327260e-05  0.132364  1.614164e-10   
1  1.465642e-08  0.003468  0.000076  5.257303e-07  0.027051  4.927366e-11   

   Surprise  
0  0.003985  
1  0.969405  
C:\Users\Stefano\workspace\EmodashSamples\SamplesStandard\AudioData\AudioData\DC\a03.wav
          Anger   Disgust      Fear     Happiness   Neutral       Sadness  \
0  2.467080e-09  0.710036  0.000187  1.752428e-08  0.087862  7.787553e-07   

   Surprise  
0  0.201915  
C:

In [15]:
import pickle

with open('baseline_mean_sd.pickle', 'rb') as f:
        data_dict = pickle.load(f)


In [16]:
data_dict

{'mean': F1      0.216773
 F2      0.013962
 F3      2.873969
 F4      0.312087
 F5      0.232803
 F6      1.673590
 F7      0.008511
 F8      0.380520
 F9    -28.298735
 F10     1.551537
 F11    -0.410844
 F12    -0.310689
 F13    -0.317822
 F14     0.020887
 F15    -0.108504
 F16    -0.058962
 F17    -0.050438
 F18    -0.079914
 F19    -0.059984
 F20    -0.059330
 F21    -0.061714
 F22     0.017673
 F23     0.008670
 F24     0.029054
 F25     0.015675
 F26     0.015916
 F27     0.013855
 F28     0.017712
 F29     0.005443
 F30     0.010193
 F31     0.013305
 F32     0.016093
 F33     0.009804
 F34     0.020049
 dtype: float64, 'sd': F1     0.052421
 F2     0.010019
 F3     0.120432
 F4     0.047327
 F5     0.022069
 F6     0.349290
 F7     0.002751
 F8     0.111443
 F9     2.748299
 F10    0.560494
 F11    0.386987
 F12    0.235608
 F13    0.225211
 F14    0.161537
 F15    0.130334
 F16    0.104909
 F17    0.094200
 F18    0.114524
 F19    0.096467
 F20    0.108135
 F21    0.109757
 